### Done
* Describe Keras

### To do
* Creating a Sequential model
* Accessing various parts of the sequential model
* Load pre-trained weights

### Notes


# Keras Fundamentals
This notebook serves as a quick introduction to Keras.
As with the TF introduction this is very quick, so have a look at the **external resources** in order to get more depth:
* [Keras code](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/keras) in the TF github repo. Looking at the code is often the best way to understand what is going on.
* [Keras as a simplified interface to TensorFlow: tutorial](https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html)
* [TensorFlow High-Level APIs: Models in a Box (TensorFlow Dev Summit 2017)](https://www.youtube.com/watch?v=t64ortpgS-E) 17 min video
* [Integrating Keras & TensorFlow: The Keras workflow, expanded (TensorFlow Dev Summit 2017)](https://www.youtube.com/watch?v=UeheTiBJ0Io) 18 min video


## High-level APIs
Pure TF is very verbose, and it is therefore a good idea to use a high-level API of some sort.
This simplifies and speeds-up development, reduces the risk of bugs, and generally reduces headache.
Another neat benefit is that a lot of best practices (initialization, scoping, etc.) are hard-coded into the functions.

There are other high-level APIs than Keras, such as 
[TFLearn](http://tflearn.org/),
[tf.learn](https://www.tensorflow.org/get_started/tflearn) (yes, the names are confusing :( ),
[tf.slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim), and
[tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers).
These are generally intercompatible, and offer overlaping functionality.
This can be confusing.
In order to simplify things we have chosen to focus on just one: Keras.

**[Keras](https://keras.io/)**
is a high-level API that can use TensorFlow or Theano as backend.
In early 2017 [TensorFlow chose Keras](http://www.fast.ai/2017/01/03/keras/) as the first high-level API to include into the tensorflow core (as opposed to contrib).
Keras has official Google support and has a large community and pre-existing examples.
Making it the best high-level API to learn at the moment (according to the author).

Keras sits of top of TFLayers, and shares its implementation (e.g. `tf.layers.dense` and `keras.layers.Dense` are the same).
Keras helps with model creation, training, and evaluation.
We won't use Keras to its full potential in this course, as it sometimes encapsulates the details to such an extent that it hinders learning.

![](keras_overview.png)



## A Basic Example

Keras can be accessed either as a stand alone library (that can use different backends), or through TF.
This only affects the import statements, and porting code back and forth should be very simple.
We will access Keras through TF, in order to ensure that the versions are fully compatible.




In [ ]:
import tensorflow as tf
import tensorflow.contrib.keras as keras
import tensorflow.contrib.keras.api.keras.models as models
import tensorflow.contrib.keras.api.keras.layers as layers

if int(tf.__version__[0]) < 1:
    print('WARNING: This will probably not work with TF version < 1.0')
    print('Your version is ' + tf.__version__)

## Using pre-trained models


Read more about [using pretrained models with Keras](https://keras.io/applications/)

22 million parameters

[Inception in TF](https://github.com/tensorflow/models/tree/master/inception)

Code: [inception_v3.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/keras/python/keras/applications/inception_v3.py). How it is implemented in Keras.

In [ ]:
## RUN THIS CELL ASAP! It takes a while to complete!
## The cell downloads the weights (~80 mb), if they aren't already.

# Create a 
inception_stem = tf.contrib.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False, # Don't include the dense layers on top of the network
    input_shape=None, # width and height should be no smaller than 139
#     pool='avg'
)

# Initially we don't want to change 
inception_stem.trainable = False

In [ ]:
## Try and 
# inception_stem.summary()

# Credits
Created by Toke Faurby ([faur](https://github.com/Faur)).
